<a href="https://colab.research.google.com/github/juancarlosmontesdeoca/Naive-Bayes-Project/blob/main/Naive_Bayes_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Análisis de sentimientos**

Los modelos Naive Bayes son muy útiles cuando queremos analizar sentimientos, clasificar textos en tópicos o recomendaciones, ya que las características de estos desafíos cumplen muy bien con los supuestos teóricos y metodológicos del modelo.

En este proyecto practicarás con un conjunto de datos para crear un clasificador de reseñas de la tienda de Google Play.

In [83]:
import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report


In [84]:
url = "https://raw.githubusercontent.com/4GeeksAcademy/naive-bayes-project-tutorial/main/playstore_reviews.csv"

In [85]:
df = pd.read_csv(url)

In [86]:
df

,package_name,review,polarity
0,com.facebook.katana,privacy at least put some option appear offli...,0
1,com.facebook.katana,"messenger issues ever since the last update, ...",0
2,com.facebook.katana,profile any time my wife or anybody has more ...,0
3,com.facebook.katana,the new features suck for those of us who don...,0
4,com.facebook.katana,forced reload on uploading pic on replying co...,0
...,...,...,...
886,com.rovio.angrybirds,loved it i loooooooooooooovvved it because it...,1
887,com.rovio.angrybirds,all time legendary game the birthday party le...,1
888,com.rovio.angrybirds,ads are way to heavy listen to the bad review...,0
889,com.rovio.angrybirds,fun works perfectly well. ads aren't as annoy...,1


**Caracteristicas de las variables**

package_name. Nombre de la aplicación móvil (categórico)

review. Comentario sobre la aplicación móvil (categórico)

polarity. Variable de clase (0 o 1), siendo 0 un comentario negativo y 1, positivo (categórico numérico)

In [87]:
df['polarity'].value_counts( )

,count
polarity,
0,584
1,307


In [88]:
df.isnull().sum()

,0
package_name,0
review,0
polarity,0


In [89]:
#Verificamos si hay nulos para eliminarlos

df.duplicated().sum()

np.int64(0)

In [90]:
# Eliminamos la columna irrelevante

df = df.drop(columns=["package_name"])

# Limpiar texto

df['review'] = df['review'].str.strip().str.lower()



In [91]:

X = df['review']
y = df['polarity']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4241234)

In [92]:

vec_model = CountVectorizer()
X_train_vec = vec_model.fit_transform(X_train)
X_test_vec = vec_model.transform(X_test) # y esto que??

CountVectorizer

Es una herramienta de scikit-learn que convierte un conjunto de textos en una matriz de frecuencias de palabras donde cada columna representa una unica palabra del vocabulario y cada fila un documento.

El valor en la celda es el numero de veces que aparece esa palabra en ese documento.

In [93]:
# MultinomialNB
model_mnb = MultinomialNB()
model_mnb.fit(X_train_vec, y_train)
y_pred_mnb = model_mnb.predict(X_test_vec)
print(classification_report(y_test, y_pred_mnb))

# BernoulliNB
model_bnb = BernoulliNB()
model_bnb.fit(X_train_vec, y_train)
y_pred_bnb = model_bnb.predict(X_test_vec)
print(classification_report(y_test, y_pred_bnb))

              precision    recall  f1-score   support

           0       0.81      0.93      0.87       119
           1       0.81      0.57      0.67        60

    accuracy                           0.81       179
   macro avg       0.81      0.75      0.77       179
weighted avg       0.81      0.81      0.80       179

              precision    recall  f1-score   support

           0       0.82      0.93      0.87       119
           1       0.82      0.60      0.69        60

    accuracy                           0.82       179
   macro avg       0.82      0.77      0.78       179
weighted avg       0.82      0.82      0.81       179



Observaciones:

Ambos modelos tienen accuracy (aciertos) muy parecidos, MultinomialNB 0.81 y BernoulliNB 0.82, este ultimo un punto mas.

Con respecto a la sensibilidad (capacidad del modelo de detectar los comentarios positivos (1) dado que el comentario es positivo), el modelo BernoulliNB obtuvo 0.60, tres puntos por encima del MultinomianlNB.

En cuanto a la especificidad (capacidad del modelo de detectar los comentarios negativos (0) dado que el comentario es negativo), ambos modelos tienen igual resultado de 0.93, muy buenos detectando la clase (0).

Si los comentarios positivos son de gran importancia, entonces se aplicara el metodo de GridSearch a efectos de mejorar la sensibilidad y asi no perder los mismos.

In [94]:

def grid_multinomialNB(X_train_vec, y_train, X_test_vec, y_test):
    param_grid = {
        'alpha': [0.1, 0.5, 1.0],   # suavizado de Laplace
        'fit_prior': [True, False]  # si aprende las probabilidades a priori
    }
    grid = GridSearchCV(MultinomialNB(), param_grid, cv=5, scoring='accuracy')
    grid.fit(X_train_vec, y_train)

    print("Mejores parámetros MultinomialNB:", grid.best_params_)
    y_pred = grid.predict(X_test_vec)
    print(classification_report(y_test, y_pred))



In [95]:
def grid_bernoulliNB(X_train_vec, y_train, X_test_vec, y_test):
    param_grid = {
        'alpha': [0.1, 0.5, 1.0],
        'fit_prior': [True, False]
    }
    grid = GridSearchCV(BernoulliNB(), param_grid, cv=5, scoring='accuracy')
    grid.fit(X_train_vec, y_train)

    print("Mejores parámetros BernoulliNB:", grid.best_params_)
    y_pred = grid.predict(X_test_vec)
    print(classification_report(y_test, y_pred))


In [96]:

grid_multinomialNB(X_train_vec, y_train, X_test_vec, y_test)
grid_bernoulliNB(X_train_vec, y_train, X_test_vec, y_test)


Mejores parámetros MultinomialNB: {'alpha': 1.0, 'fit_prior': True}
              precision    recall  f1-score   support

           0       0.81      0.93      0.87       119
           1       0.81      0.57      0.67        60

    accuracy                           0.81       179
   macro avg       0.81      0.75      0.77       179
weighted avg       0.81      0.81      0.80       179

Mejores parámetros BernoulliNB: {'alpha': 1.0, 'fit_prior': True}
              precision    recall  f1-score   support

           0       0.82      0.93      0.87       119
           1       0.82      0.60      0.69        60

    accuracy                           0.82       179
   macro avg       0.82      0.77      0.78       179
weighted avg       0.82      0.82      0.81       179



**Observaciones**

-La grilla no produjo mejoras significativas, en ambos casos BernoulliNB y MultinomialNB) respecto a los valores obtenidos sin hiperparametros.

-Esto puede deberse a que los parámetros MultinomialNB: {'alpha': 1.0, 'fit_prior': True} y parámetros BernoulliNB: {'alpha': 1.0, 'binarize': 0.0, 'fit_prior': True} no generaron cambios sustanciales en el comportamiento del modelo, por lo tanto se probara agregando pesos a la clase 0 y la clase 1.

In [97]:
def grid_multinomialNB(X_train_vec, y_train, X_test_vec, y_test):
    param_grid = {
        'alpha': [0.1, 0.5, 1.0],   # suavizado de Laplace
        'fit_prior': [True, False], # si aprende las probabilidades a priori
        'class_prior': [
            None,                   # usa las priors aprendidas de los datos
            [0.5, 0.5],             # peso balanceado
            [0.4, 0.6],             # mas peso a la clase 1 (positivos)
            [0.3, 0.7],             # aun mas  peso a la clase 1
            [0.6, 0.4]              # mas peso a la clase 0 (negativos)
        ]
    }
    grid = GridSearchCV(MultinomialNB(), param_grid, cv=5, scoring='accuracy')
    grid.fit(X_train_vec, y_train)

    print("Mejores parámetros MultinomialNB:", grid.best_params_)
    y_pred = grid.predict(X_test_vec)
    print(classification_report(y_test, y_pred))


In [100]:

def grid_bernoulliNB(X_train_vec, y_train, X_test_vec, y_test):
    param_grid = {
        'alpha': [0.1, 0.5, 1.0],
        'fit_prior': [True, False],
        'class_prior': [
            None,
            [0.5, 0.5],
            [0.4, 0.6],
            [0.3, 0.7],
            [0.6, 0.4]
        ]
    }
    grid = GridSearchCV(BernoulliNB(), param_grid, cv=5, scoring='accuracy')
    grid.fit(X_train_vec, y_train)

    print("Mejores parámetros BernoulliNB:", grid.best_params_)
    y_pred = grid.predict(X_test_vec)
    print(classification_report(y_test, y_pred))

In [101]:
grid_multinomialNB(X_train_vec, y_train, X_test_vec, y_test)
grid_bernoulliNB(X_train_vec, y_train, X_test_vec, y_test)

Mejores parámetros MultinomialNB: {'alpha': 1.0, 'class_prior': [0.4, 0.6], 'fit_prior': True}
              precision    recall  f1-score   support

           0       0.84      0.92      0.88       119
           1       0.80      0.65      0.72        60

    accuracy                           0.83       179
   macro avg       0.82      0.78      0.80       179
weighted avg       0.82      0.83      0.82       179

Mejores parámetros BernoulliNB: {'alpha': 1.0, 'class_prior': None, 'fit_prior': True}
              precision    recall  f1-score   support

           0       0.82      0.93      0.87       119
           1       0.82      0.60      0.69        60

    accuracy                           0.82       179
   macro avg       0.82      0.77      0.78       179
weighted avg       0.82      0.82      0.81       179



**Conclusiones**

Los modelos son muy buenos detectando los comentarios negativos (clase 0), sin embargo tienen dificultades para detectar comentarios positivos (clase 1), ya que la sensibilidad ronda entre 0.57 y 060.

Ahora bien, ajustandole pesos a la clase 0 (0.4) y clase 1 (0.6), el Modelo MultinomialNB ofrece un mejor balance entre precision y sensibilidad en ambas clases, lo cual es util para no perder tantos comentarios positivos.

